In [53]:
!pip3 install tradingview-ta

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from tradingview_ta import TA_Handler, Interval, Exchange


In [11]:
output = TA_Handler(symbol = 'BTCUSDT',
                    screener = 'Crypto',
                    exchange = 'Binance',
                    interval = Interval.INTERVAL_15_MINUTES)

In [14]:
output.get_analysis().summary

{'RECOMMENDATION': 'BUY', 'BUY': 11, 'SELL': 6, 'NEUTRAL': 9}

In [35]:
output.get_analysis().summary['RECOMMENDATION']

'NEUTRAL'

In [58]:
engine = create_engine("sqlite:///LivePrice.db")

In [59]:
df = pd.read_sql('BTCUSDT',engine)

In [60]:
df2 = df.resample(rule='15Min', on='timestamp').mean()
df2

,price
timestamp,
2022-06-05 18:00:00,30040.586734
2022-06-05 18:15:00,30038.675674
2022-06-05 18:30:00,30003.372374
2022-06-05 18:45:00,30014.955571


In [61]:
in_position = False

while True:
    df = pd.read_sql('BTCUSDT', engine)
    last_Price = df.price.iloc[-1]
    if not in_position:
        if last_Price < df.price.mean() and 'BUY' == output.get_analysis().summary['RECOMMENDATION']:# 平均值會有問題 db的平均值不準因為db沒有24小時的資料,交易也不需要，需取30m~2hr 平均值        
            buyprice = last_Price
            
            in_position = True
            print(df.price.mean())
            print("Bought for " , buyprice )

    if in_position:
        if last_Price > buyprice * 1.003 and 'SELL' == output.get_analysis().summary['RECOMMENDATION'] or last_Price < buyprice * 0.9997 :
            sellprice = last_Price
            in_position = False
            print('Sold for ' , sellprice)
            print('profit is ' , (sellprice - buyprice))

            
        


30023.994743777454
Bought for  30004.34


KeyboardInterrupt: 